In [1]:
# this notebook is used to explore the data we have around diagnosis

In [2]:
# cross check dates of first azotemia, low USG with diagnoses
# come up with scheme for first diagnosed date
# generally look at what diagnosis info we have
# add a column to diagnose based on IRIS reference ranges
# add a column for Rachel's diagnosis reference range for species she calculated

In [3]:
# 'Date of Onset of Azoatemia'  completely empty
# 'Date of USG less than 1,035' empty
# 'Onset of azotaemia' these are dates and there are 56 of them

In [4]:
# SCHEME FOR INFERRING DATE OF DIAGNOSIS
# logic_flag
# 0: can't infer
# 1: date of diagnosis (4 cats)
# 2. 'First report of CKD CS' (7)
# 3. Ante-mortem Dx TRUE and use earliest 'Onset of azotaemia' / 'Date of first low USG'
# 4. Died of CKD  and use earliest 'Onset of azotaemia' / 'Date of first low USG'

# sometimes the first indicator date is the same as date of death.  Once labeling is done will need to compare date of death and fisrt indicator date and consider excluding record if these are too close in time.


In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
# read in unified animal list
file = '../Data/big_cats/Access_DB_table_exports/PrevalenceIncludedFelidsTZ_ZV_WA_SA_MG_DM.xlsx'
data = pd.read_excel(file)

In [ ]:
data.columns

In [ ]:
diagnosis_date_labeling = data[['ARKS Number', 'Ante-mortem Dx of CKD', 'Date of first low USG', 'Onset of azotaemia', 'Date of diagnosis', 'First report of CKD CS', 'Cause of Death', 'CKD - cause of mortality', 'Date of Death']]
diagnosis_date_labeling

In [ ]:
# write to csv to label
#file = '../Data/big_cats/processed_data/diagnosis_date_labeling.csv'
#diagnosis_info.to_csv(file, index=False)

In [ ]:
# SCHEME FOR INFERRING DATE OF DIAGNOSIS
# logic_flag
# 0: can't infer
# 1: date of diagnosis (4 cats)
# 2. 'First report of CKD CS' (7)
# 3. Ante-mortem Dx TRUE and use earliest 'Onset of azotaemia' / 'Date of first low USG'
# 4. Died of CKD  and use earliest 'Onset of azotaemia' / 'Date of first low USG'

In [ ]:
data['inferred_diagnosis_date'] = None
data['logic_flag'] = 0

logic_flag_4 =(data['CKD - cause of mortality']) & ((data['Onset of azotaemia'].notnull()) | (data['Date of first low USG'].notnull()) )
data.loc[logic_flag_4, 'logic_flag'] = 4
data.loc[logic_flag_4, 'inferred_diagnosis_date'] = data[logic_flag_4][['Onset of azotaemia', 'Date of first low USG']].min(axis=1)

logic_flag_3 =(data['Ante-mortem Dx of CKD']) & ((data['Onset of azotaemia'].notnull()) | (data['Date of first low USG'].notnull()) )
data.loc[logic_flag_3, 'logic_flag'] = 3
data.loc[logic_flag_3, 'inferred_diagnosis_date'] = data[logic_flag_3][['Onset of azotaemia', 'Date of first low USG']].min(axis=1)

logic_flag_2 = data['First report of CKD CS'].notnull()
data.loc[logic_flag_2, 'logic_flag'] = 2
data.loc[logic_flag_2, 'inferred_diagnosis_date'] = data[logic_flag_2]['First report of CKD CS']

logic_flag_1 = data['Date of diagnosis'].notnull()
data.loc[logic_flag_1, 'logic_flag'] = 1
data.loc[logic_flag_1, 'inferred_diagnosis_date'] = data[logic_flag_1]['Date of diagnosis']

data['inferred_diagnosis_date'] = pd.to_datetime(data['inferred_diagnosis_date'])

data['earliest_indicator'] = data[['Onset of azotaemia', 'Date of first low USG', 'inferred_diagnosis_date']].min(axis=1)
data['time_between_diagnosis_and_death'] = data['Date of Death'] - data['inferred_diagnosis_date']
data['time_between_earliest_indicator_and_death'] = data['Date of Death'] - data['earliest_indicator']


In [ ]:
data.head()

In [ ]:
data['CKD - cause of mortality'].value_counts()

In [ ]:
CKD = data[(data['CKD - cause of mortality']==True) & (data['logic_flag']!=0)]

In [ ]:
len(CKD)

In [ ]:
CKD.loc[67] # cats still alive: 27, 42, 52, 65

In [ ]:
CKD['Date of Death'] - CKD['inferred_diagnosis_date']

In [ ]:
CKD[['time_between_diagnosis_and_death', 'time_between_earliest_indicator_and_death']]

In [ ]:
data.dtypes

In [ ]:
len(data['ARKS Number'].unique())

In [ ]:
data['Ante-mortem Dx of CKD'].value_counts()

In [ ]:
len(data[(data['CKD - cause of mortality'] == True) & (data['Ante-mortem Dx of CKD'] == True)])

In [ ]:
data['Date of diagnosis'].value_counts()

In [ ]:
data['First report of CKD CS'].value_counts()

In [ ]:
data['Onset of azotaemia'].value_counts().sum()

In [ ]:
data['Date of first low USG'].value_counts().sum()

In [ ]:
data['Date of USG less than 1,035'].value_counts()

In [ ]:
data['Date of Onset of Azoatemia'].value_counts()